# BiVital Data Processing Example
This notebook demonstrates how to use the BiVital library for processing data. 
It covers data loading and interpolation with practical examples.

## Import the processing lib from bivtial and set up logging 

In [ ]:
from bivital import processing as bvl
from tkinter import filedialog, Tk
import logging

#Set up logging
logging.basicConfig(encoding='utf-8', level=logging.INFO)

## Choose the project directory for loading

In [ ]:
#Choose the example project data or choose your own directory
# 0 = example data
# 1 = add your path 
# else = Choose Directory --> This will open a file dialog for you to choose the directory, check the opened windows in the background

Data_to_use = 0

if Data_to_use == 0:
    project_directory = bvl.path_to_example_data()
else:
    project_directory = "Path/to/your/BiVital/Project"  # Replace with the path to your data
    # macOS/Linux example: "/home/you/Documents/BI_Vital_Projects/Bi_Vital_Tutorial"
    # Windows example:     "C:/Users/you/Documents/BI_Vital_Projects/Bi_Vital_Tutorial"

## Load the data

In [ ]:
project_data = bvl.ProjectData(project_directory)

## Interpolate

In this Example the interpoaltion will be used directly on the loaded data without any preprocessing. 

The interpolation function has three parameters:
- `method`: Interpolation technique to use. [Default = linear]
    - `linear`
    - `nearest` 
    - (for more check the method paramter here: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html)
- `inplace`: Update the data in place. [Default = False]
- `region`: Data for processing. [Default = whole project]
    - `[]`: whole project
    - `[series, dataset, column]`: specific data

In [ ]:
interpolated_data = project_data.interpolate(method="linear", inplace=False, region=[])

### Visualize the result

In [ ]:
from IPython.display import display_html

def display_side_by_side(df1, df2, start_row=20, num_rows=50, names=['Original', 'Interpolated']):
    # Get the slice of data from start_row to start_row + num_rows
    df1_slice = df1.iloc[start_row:start_row + num_rows]
    df2_slice = df2.iloc[start_row:start_row + num_rows]
    
    html_str = f'''<div style="display:flex">
                   <div style="flex:1">
                     <h3>{names[0]} (Rows {start_row}-{start_row + num_rows})</h3>
                     {df1_slice.to_html()}
                   </div>
                   <div style="flex:1">
                     <h3>{names[1]} (Rows {start_row}-{start_row + num_rows})</h3>
                     {df2_slice.to_html()}
                   </div>
                   </div>'''
    display_html(html_str, raw=True)

# Compare rows 200-300 of original and interpolated data
display_side_by_side(project_data[0,0], interpolated_data[0,0], start_row=200, num_rows=50)

As demonstrated in the comparison above, the interpolation process has successfully filled the missing values (marked as `<NA>` and `NaN`) with estimated data points. This improvement is particularly visible in the plots below, where the interpolated data shows a continuous signal pattern instead of gaps from dropped `NaN` entries. The linear interpolation creates a more complete dataset by estimating values between known data points, resulting in a smoother and more detailed representation of the original measurements.

In [ ]:
import matplotlib.pyplot as plt

# Create two vertically stacked subplots (2 rows, 1 column)
fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

# Plot ECG data on the first subplot
project_data[0, 0, "Heartrate", 200:250].plot(
    ax=axes[0],                 # Use the first Axes object (top subplot)
    grid=True,                  # Show grid lines for better readability
    title="Heartrate Example Data",   # Set plot title
    xlabel="",                  # Leave x-axis label empty (set it on the bottom subplot)
    ylabel="Amplitude",         # Label for the y-axis
    style = '.'
)
axes[0].tick_params(axis='x', rotation=45)  # Rotate x-axis tick labels to avoid overlap

interpolated_data[0, 0, "Heartrate", 200:250].plot(
    ax=axes[1],                 # Use the first Axes object (top subplot)
    grid=True,                  # Show grid lines for better readability
    title="Heartrate Example Data",   # Set plot title
    xlabel="Time",                  # Label for the x-axis
    ylabel="Amplitude",         # Label for the y-axis
    style = '.'
)
axes[1].tick_params(axis='x', rotation=45)  # Rotate x-axis tick labels to avoid overlap


# Automatically adjust subplot layout to prevent overlap
plt.tight_layout()

# Display the final combined figure
plt.show()